# Predict TripAdvisor Rating

Выполнение задания:
1. Анализ исходных данных;
2. Очистка и подготовка данных - очистка от мусора, работа с nan, перекодирование номинальных признаков;
3. Создание новых признаков (Feature Engineering) на основе имеющихся данных;
4. EDA: проведение анализа числовых и категориальных данных, построение графиков
5. ML: проверка очищенных данных, запуск модели ML, прогноз и MAE
6. Исследование признаков оказавших наиболее сильное влияние на результат прогноза
7. Submission

# libraries import

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
    
import re
from datetime import datetime, timedelta
from collections import Counter
import pandas_profiling

# Any results you write to the current directory are saved as output.

In [2]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [4]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [5]:
df_train.info()

In [6]:
df_train.head(5)

In [7]:
df_test.info()

In [8]:
df_test.head(5)

In [9]:
sample_submission.head(5)

In [10]:
sample_submission.info()

In [11]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [12]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [13]:
data.sample(5)

In [14]:
# для удобства причешем название признаков
data.columns = list(map(lambda x: x.replace(" ","_"), data.columns.str.lower()))
data.info()

In [15]:
# сразу удаляем признак url_ta так как мы не будем использовать внешние данные с сайта TripAdvisor
data = data.drop('url_ta',axis=1)
data

# ОЧИСТКА И РАБОТА С ПРИЗНАКАМИ
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [16]:
# процентное соотношение пропусков для каждого признака
# ВЫВОД: признак price_range содержит более 30% пропусков - по правилам его следует исключить (пока оставлю)
for col in data.columns:
    pct_missing = np.mean(data[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

In [17]:
# проверяем наличие списка в данных типа object
# ВЫВОД: несмотря на наличие квадратных скобок - списков в датасете НЕТ
l = data.sample(2).applymap(type).astype(str) == "<class 'list'>"
l

In [18]:
# создаем новые признаки для значений NaN
data['cuisine_style_isNaN'] = pd.isna(data['cuisine_style']).astype('uint8')
data['price_range_isNaN'] = pd.isna(data['price_range']).astype('uint8')
data['number_of_reviews_isNaN'] = pd.isna(data['number_of_reviews']).astype('uint8')

# визуально в признаке reviews есть и пропуски NaN, и значения вида '[[], []]'.
# приведем их к единому виду '[[], []]' и сбросим эти значения в новый признак
data['reviews'] = data['reviews'].fillna('[[], []]')
data['reviews_isNaN'] = data['reviews'].apply(lambda x: int(x ==  '[[], []]'))

In [19]:
data.info()

In [20]:
data.nunique()

#### restaurant_id

In [21]:
# убираем приставку id_
data['restaurant_id'] = data.restaurant_id.str.replace('id_', '')
# преобразовываем restaurant_id тип данных из object в int
data['restaurant_id'] = data['restaurant_id'].astype(str).astype(int)

#### cuisine_style

In [22]:
# заменяем пропуски NaN на значение 'regional'
data['cuisine_style'] = data['cuisine_style'].fillna('regional')
# чистим данные, заменяя '[' и ']' на пробелы
data['cuisine_style'] = [x.replace(']', '') for x in data['cuisine_style']]
data['cuisine_style'] = [x.replace('[', '') for x in data['cuisine_style']]

# объединяем названия кухонь в единый список с помощью str.cat (конкатенация) 
# потом вновь делим по отдельным значениям кухонь, создаем серию и считаем количество каждой кухни
cuisine_list = pd.Series(data['cuisine_style'].str.cat(sep=', ').split(', ')).value_counts()
cuisine_list

In [23]:
# создаем новый признак для указания количества кухонь по каждому ресторану
data['cuisine_count'] = [len(x.split(', ')) for x in data['cuisine_style']]

#### price_range

In [24]:
# количество каждого из значений по колонке Price Range
# заменяем значение NaN на 'no data'
data['price_range'] = data['price_range'].fillna('no data')
data['price_range'].value_counts(dropna=False).sort_index(ascending=True)
# создаем словарь замены номинальных значений на количественные
# присваиваем значению 'no data' параметр 2 равные категории '$$ - $$$' как наиболее часто встречающейся
range_dict = {'$$$$': 3, '$$ - $$$': 2, '$': 1, 'no data': 2}
# заменяем номинальные значения в Price Range на значения из созданного словаря
data['price_range'] = data['price_range'].replace(to_replace = range_dict)

#### number_of_reviews

In [25]:
# заменяем пропуски NaN средним арифметическим значением 
replace = np.round(data['number_of_reviews'].mean())
data['number_of_reviews'] = data['number_of_reviews'].fillna(replace)

#### reviews

In [26]:
# мы хотим: вытащить дату первого и второго отзывов, посчитать между ними разницу
# вытаскиваем дату первого и второго отзыва
date_1 = data['reviews'].str[-27:-17]
date_2 = data['reviews'].str[-13:-3]

In [27]:
# приводим обе даты к формату datetime
date_1 = pd.to_datetime(date_1, format = '%m/%d/%Y', errors='coerce')
date_2 = pd.to_datetime(date_2, format = '%m/%d/%Y', errors='coerce')

In [28]:
# считаем разницу между отзывами и берем значение по модулю, так как даты могут быть перепутаны
# тогда вылезет отрицательное значение дельты (абсолютные цифры будут одинаковыми)
dates_gap = abs(date_1 - date_2)
# из типа timedelta приводим к типу int и заполняем пропуски NaN нулями
dates_gap = pd.to_numeric(dates_gap.dt.days, downcast='integer').fillna(0)
dates_gap

In [29]:
# пробуем вытащить из текста отзывов ключевые слова для определения оттенка отзыва
# создаем два вида словаря и помещаем выборку ключевых слов (определили визуально по тексту)
positive = ['excellent', 'great', 'perfect', 'exceptional', 'best', 'unique',
            'brilliant', 'lovely', 'outstanding', 'amazing', 'delicious', 'very good',
            'awesome', 'fabulous', 'super', 'fantastic', 'gem', 'superb','good', 
            'nice', 'tasty', 'beautiful', 'fresh', 'yummy', 'cool']
negative = ['not bad', 'simple', 'not exceptional', 'poor''bad', 'wasting', 'disappointment', 
            'burnt food', 'refund', 'do not eat',
            'disgusting', 'awful', 'poor', 'limited', 'down', 'avoid', 'did not enjoy',
            'not ok', 'horrible', 'worst', 'terrible']

#### id_ta

In [30]:
# убрали первый символ d и сделали целым числом
data['id_ta'] = data['id_ta'].apply(lambda x: x[1:]).astype(int)

### 2. Cоздание новых признаков (Feature Engenearing)

In [31]:
# количество жителей по городам
population = {'Paris': 2140526, 'Stockholm': 961609, 'London': 8787892, 
              'Berlin': 3601131, 'Munich': 1456039, 'Oporto': 221800, 
              'Milan': 1366180, 'Bratislava': 437725, 'Vienna': 1840573, 
              'Rome':2872800, 'Barcelona':  1620343, 'Madrid': 3223334,
              'Dublin' : 553165, 'Brussels' : 1198726, 'Zurich' : 434008, 
              'Warsaw' : 1702139, 'Budapest' : 1752286, 'Copenhagen' : 615993,
              'Amsterdam' : 859732, 'Lyon' : 515695, 'Hamburg' : 1830584, 
              'Lisbon': 553000, 'Prague' : 1280508, 'Oslo' : 673469,
              'Helsinki' : 643272, 'Edinburgh': 513210 , 'Geneva': 201818, 
              'Ljubljana' : 284355, 'Athens' : 655780, 'Luxembourg' : 122273,
              'Krakow' : 779115}


data['population'] = data['city'].apply(lambda x: population[x])

In [32]:
# количество туристов по городам

tourists = {'Paris': 17560200, 'Stockholm': 2604600, 'London': 19233000, 
              'Berlin': 5959400, 'Munich': 4066600, 'Oporto': 2341300, 
              'Milan': 6481300, 'Bratislava': 1285200, 'Vienna': 6410300, 
              'Rome': 10065400, 'Barcelona':  6714500, 'Madrid': 5440100,
              'Dublin' : 5213400, 'Brussels' : 3942000, 'Zurich' : 2240000, 
              'Warsaw' : 2850000, 'Budapest' : 3822800, 'Copenhagen' :3069700,
              'Amsterdam' : 8354200, 'Lyon' : 6000000, 'Hamburg' : 1450000, 
              'Lisbon': 3539400, 'Prague' : 8948600, 'Oslo' : 1300000,
              'Helsinki' : 4167982, 'Edinburgh': 1660000, 'Geneva': 1150000, 
              'Ljubljana' : 1300000, 'Athens' : 5728400, 'Luxembourg':1300000,
              'Krakow' : 2732000}

data['tourists'] = data['city'].apply(lambda x: tourists[x])

In [33]:
# количество ресторанов в каждом городе
restorants_in_city = data.groupby(data.city)['ranking'].count().to_dict()
data['restaurants_in_city'] = data.city.map(restorants_in_city)

# количество ресторанов на 1000 горожан
rests_for_citizens = data['restaurants_in_city'] / (data['population']*1000)
# количество ресторанов на 10000 туристов
rests_for_tourists = data['restaurants_in_city'] / (data['tourists']*10000)

data['rests_for_citizens'] = rests_for_citizens
data['rests_for_turistis'] = rests_for_tourists

In [34]:
# средний рейтинг ресторана
data['average_rank'] = data['ranking'] / data['restaurants_in_city']

In [35]:
# среднее количество кухонь по ресторанам в городе
average_cousines = round(data.groupby(data.city)['cuisine_count'].mean()).to_dict()
data['average_cousines'] = data.city.map(average_cousines)

In [36]:
# наличие в ресторане одной из 3-х самых популярных кухонь (кроме Regional = NaN)
popular_cuisine = ['Vegetarian Friendly', 'European', 'Mediterranian']
data['popular_cuisine'] = data['cuisine_style'].apply(lambda x: int(any(word in x for word in popular_cuisine)))

In [37]:
# признаки оттенка текста отзыва
data['positive_review'] = data['reviews'].apply(lambda x: int(any(word in x for word in positive)))
data['negative_review'] = data['reviews'].apply(lambda x: int(any(word in x for word in negative)))

# EDA 
Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.

### Посмотрим распределение признака

In [38]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [39]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [40]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [41]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.


In [42]:
# нормируем ранг ресторанов по городам
mean_Ranking_on_City = data.groupby(['city'])['ranking'].mean()
max_Ranking_on_City = data.groupby(['city'])['ranking'].max()
data['max_Ranking_on_City'] = data['city'].apply(lambda x: max_Ranking_on_City[x])
data['mean_Ranking_on_City'] = data['city'].apply(lambda x: mean_Ranking_on_City[x])
data['norm_Ranking_on_maxRank_in_City'] = (data['ranking']) / (data['max_Ranking_on_City'])

In [43]:
# посмотрим на топ 10 городов
for x in (data['city'].value_counts())[0:10].index:
    data['norm_Ranking_on_maxRank_in_City'][data['city'] == x].hist(bins=100)
plt.show()

### Посмотрим распределение целевой переменной

In [44]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [45]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [46]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### Корреляция признаков
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [47]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

### Отсмотр коллекции признаков

In [48]:
data.hist(figsize=(20, 20), bins=100);
plt.tight_layout()

In [49]:
plt.rcParams['figure.figsize'] = (15,15)
sns.heatmap(data.corr(), square=True, annot=True, fmt=".1f", linewidths=0.1,
                                                                cmap="RdBu");
plt.tight_layout()

In [50]:
data.corr()

### DUMMIES

In [51]:
# применяем для признаков с типом object: 'city'
data = pd.concat([data, pd.get_dummies(data['city'].apply(pd.Series).stack())
                                                      .sum(level=0)], axis=1)

In [52]:
# удаляем столбцы признаков object, которые мы уже использовали для Feature Engenearing
data.drop(['city', 'cuisine_style', 'reviews'], axis=1, inplace=True)

#### Запускаем и проверяем что получилось

In [53]:
df_preproc = data
df_preproc.sample(10)

In [54]:
# выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.rating.values            # наш таргет
X = train_data.drop(['rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [55]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [56]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [57]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [58]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [59]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [60]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [61]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [62]:
test_data.sample(10)

In [63]:
test_data = test_data.drop(['rating'], axis=1)

In [64]:
sample_submission

In [65]:
predict_submission = model.predict(test_data)

In [66]:
# для удобства сравнения делаем рейтинги кратными 0.5
predict_submission = np.round(predict_submission * 2) / 2
predict_submission

In [67]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
